In [19]:

import math
import numpy as np
from copy import deepcopy

In [26]:

class RaceTrajEnv():



    def __init__(self, num_of_gates = 10, angle_restriction = math.pi*3/5, 
                        next_distance_low = 5, next_distance_high = 15, 
                        look_ahead = 1, look_behind = 1,
                        gate_width = 1, gate_height = 0.5,
                        angle_dev=0.1):

        self.num_of_gates = num_of_gates
        self.angle_restriction = angle_restriction
        self.next_distance_low = next_distance_low
        self.next_distance_high = next_distance_high
        self.look_ahead = look_ahead
        self.look_behind = look_behind
        self.gate_width = gate_width
        self.gate_height = gate_height
        self.angle_dev = angle_dev

# Action space first axis is for inplane movement and second for vertical 


        self.viewer = None
        self.state = None
        self.state_i = 0
        self.entire_traj = None
        self.entire_traj_act = None
        self.steps_beyond_done = None


    def step(self, action):

        self.entire_traj_act[self.state_i] = action
        self.state_i += 1
        done = self.state_i==len(self.entire_traj)-2

        # TODO: Calculate reward by refitting trajectory that satisfies the snap requirement
        reward = 1.0
        # Would need time based optimization?
        
        self.state = np.array(self.entire_traj[self.state_i-self.look_behind:self.state_i+self.look_ahead+1])

        return np.array(self.state), reward, done, {}

    def get_next_pt(self, cpt, ppt):
        grad = cpt - ppt
        thetad = math.atan2(grad[1], grad[0])
        gdp = math.sqrt(grad[0]**2+grad[1]**2)
        phid = math.atan2(grad[2], gdp)
        thetaf = thetad + np.clip(np.random.standard_normal()*self.angle_dev*self.angle_restriction/2, -self.angle_restriction/2, self.angle_restriction/2)
        phif = phid + np.clip(np.random.standard_normal()*self.angle_dev*self.angle_restriction/2, -self.angle_restriction/2, self.angle_restriction/2)
        npt = np.array([math.cos(thetaf), math.sin(thetaf), math.tan(phif)])
        npt = cpt + npt/np.linalg.norm(npt)*(np.random.uniform(math.sqrt(self.next_distance_low), math.sqrt(self.next_distance_high))**2)
        return npt


    def reset(self):

        self.entire_traj = [np.array([0, -1, 0]), np.array([0, 0, 0])]

        for i in range(self.num_of_gates+1):
            self.entire_traj.append(self.get_next_pt(self.entire_traj[-1], self.entire_traj[-2]))

        self.entire_traj = np.array(self.entire_traj)
        self.entire_traj_act = deepcopy(self.entire_traj)

        self.state_i = 2
        self.state = np.array(self.entire_traj[self.state_i-self.look_behind:self.state_i+self.look_ahead+1])

        self.steps_beyond_done = None
        
        return np.array(self.state)

In [27]:
env = RaceTrajEnv()

In [28]:
env.reset()

array([[ 0.        ,  0.        ,  0.        ],
       [-0.64211364,  5.00250132, -0.55891408],
       [-3.33150354, 16.23850905, -1.76358475]])

In [29]:
env.entire_traj

array([[  0.        ,  -1.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [ -0.64211364,   5.00250132,  -0.55891408],
       [ -3.33150354,  16.23850905,  -1.76358475],
       [ -6.98559302,  29.68153331,  -3.46708576],
       [ -9.84356731,  42.90126694,  -6.94124628],
       [-11.51304279,  48.58023258,  -8.03088097],
       [-14.40542896,  58.33948167,  -7.88604932],
       [-16.21922458,  68.37314264,  -7.83915805],
       [-17.28856767,  75.66642765,  -7.28021501],
       [-18.91440875,  82.99210775,  -7.74474225],
       [-20.52700488,  88.07285804,  -7.06003134],
       [-22.62544654,  93.91466485,  -6.53573441]])

In [94]:
import traj_gen
import numpy as np

from traj_gen import poly_trajectory as pt

import time

def get_knots(waypoints, scale = 10):
    total_time = 0
    for wpa, wpb in zip(waypoints[1:], waypoints[:-1]):
        total_time += np.linalg.norm(wpa-wpb)    

    knots = np.zeros((len(waypoints)))
    knots[0] = 0
    for i, (wpa, wpb) in enumerate(zip(waypoints[1:], waypoints[:-1])):
        knots[i+1] = knots[i]+np.linalg.norm(wpa-wpb)/total_time
    
    return (knots*scale).astype(np.float)


def get_trajectory_snap(waypoints, tdelta = 0.1, time_between_gates = 2):
    dim = 3
    knots = get_knots(waypoints, time_between_gates*len(waypoints))
    order = 8
    optimTarget = 'poly-coeff' #'end-derivative' 'poly-coeff'
    maxConti = 4
    objWeights = np.array([0, 0, 0, 0, 1])
    pTraj = pt.PolyTrajGen(knots, order, optimTarget, dim, maxConti)

        # 2. Pin
    Xdot = np.array([0, 0, 0])
    Xddot = np.array([0, 0, 0])

    pin_ = {'t':0, 'd':1, 'X':Xdot,}
    pTraj.addPin(pin_)
    pin_ = {'t':0, 'd':2, 'X':Xddot,}
    pTraj.addPin(pin_)


    for i, wp in enumerate(waypoints):

        pin_ = {'t':knots[i], 'd':0, 'X':wp[i, :]}
        pTraj.addPin(pin_)

    # solve
    pTraj.setDerivativeObj(objWeights)
    pTraj.solve()
    rng = np.linspace(0, len(waypoints)*time_between_gates, int((len(waypoints))*time_between_gates//tdelta))
    snap = pTraj.eval(rng, 4)
    return np.sum(snap)

def calc_bonus(waypoints, delta, gate_width = 1, gate_height = 0.5,):
    delta[:, 1] = np.clip(delta[:, 1], -gate_height/2, gate_height/2)
    delta[:, 0] = np.clip(delta[:, 0], -gate_width/2, gate_width/2)

    dvx = np.zeros((waypoints.shape[0], 3))
    dvx[1:] = waypoints[1:]-waypoints[:-1]
    dvx[0] = waypoints[1]
    az1 = np.array([0,0,1])
    az0 = np.cross(az1, dvx)
    az0 = az0/np.linalg.norm(az0)
    az1 = np.array([az1])

    waypoints_modif = waypoints.copy() + az1*delta[:, 1:] + az0* delta[:, :1]
    print(waypoints)
    print((az0* delta[:, :1]).shape)
    print(get_trajectory_snap(waypoints))
    print(get_trajectory_snap(waypoints_modif))

In [95]:
waypoints = env.entire_traj[1:]

In [96]:
waypoints.shape

(12, 3)

In [97]:
calc_bonus(waypoints, np.zeros((waypoints.shape[0], 2)))

[[  0.           0.           0.        ]
 [ -0.64211364   5.00250132  -0.55891408]
 [ -3.33150354  16.23850905  -1.76358475]
 [ -6.98559302  29.68153331  -3.46708576]
 [ -9.84356731  42.90126694  -6.94124628]
 [-11.51304279  48.58023258  -8.03088097]
 [-14.40542896  58.33948167  -7.88604932]
 [-16.21922458  68.37314264  -7.83915805]
 [-17.28856767  75.66642765  -7.28021501]
 [-18.91440875  82.99210775  -7.74474225]
 [-20.52700488  88.07285804  -7.06003134]
 [-22.62544654  93.91466485  -6.53573441]]
(12, 3)


UnboundLocalError: local variable 'total_time' referenced before assignment